<a href="https://colab.research.google.com/github/ereinha/PMAE/blob/main/UCI_Higgs_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone PMAE

In [ ]:
%cd PMAE

In [ ]:
!pwd

In [ ]:
import train
import utils
import test
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import json
from models import classifier, pmae
import torch

In [ ]:
# Download the files from UCI ML repository
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz
!gzip -d HIGGS.csv.gz

In [ ]:
# Read in the csv
data = pd.read_csv('./HIGGS.csv', header=None)

In [ ]:
# Split the data and labels
X = data.iloc[:,1:]
y = data.iloc[:,0]

In [ ]:
# Get the inputs and outputs
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

In [ ]:
print(X.shape)

In [ ]:
# Remove high level features and add 0 padding to line up related quantities
X = np.delete(X, obj=np.s_[X.shape[1]-7:X.shape[1]], axis=-1)
X = np.insert(X, obj=np.s_[3:4], values=0, axis=-1)
X = np.insert(X, obj=np.s_[5:6], values=0, axis=-1)
X = np.insert(X, obj=np.s_[7:8], values=0, axis=-1)

In [ ]:
# Visualizing the data
names = ["lepton pT", "lepton eta", "lepton phi", "Zero Padding",
         "missing energy magnitude", "Zero Padding", "missing energy phi", "Zero Padding",
         "jet 1 pt", "jet 1 eta", "jet 1 phi", "jet 1 b-tag",
         "jet 2 pt", "jet 2 eta", "jet 2 phi", "jet 2 b-tag",
         "jet 3 pt", "jet 3 eta", "jet 3 phi", "jet 3 b-tag",
         "jet 4 pt", "jet 4 eta", "jet 4 phi", "jet 4 b-tag"]

fig, axes = plt.subplots(6,4, figsize=(15,20))
fig.tight_layout(pad=4)

X_higgs = X[y==1]
X_nohiggs = X[y==0]
y_higgs = y[y==1]
y_nohiggs = y[y==0]

for i in range(len(axes)):
    for j in range(len(axes[0])):
        axes[i,j].hist(X_higgs[:,i*4+j], histtype='step', stacked=False, density=1, bins=30, label='Higgs')
        axes[i,j].hist(X_nohiggs[:,i*4+j], histtype='step', stacked=False, density=1, bins=30, label='Background')
        axes[i,j].set_title(names[i*4+j])
        axes[i,j].set_xlabel(names[i*4+j] + ' scaled')
        axes[i,j].legend()
        axes[i,j].set_ylabel("Proportion of data (density=1)")
plt.show()

In [ ]:
print(len(X_higgs))
print(len(X_nohiggs))

In [ ]:
# Create data labels for Higgs and Non-Higgs (ttbar) data (for the UCI set we can also use the y values)
labels_hh = np.ones(len(X_higgs))
labels_tt = np.zeros(len(X_nohiggs))

In [ ]:
# Split the Higgs data into train and test sets .8, .1, .1
X_hh_train, X_val1, labels_hh_train, labels_val1 = train_test_split(X_higgs, labels_hh, test_size=0.2, random_state=42)
X_hh_val, X_hh_test, labels_hh_val, labels_hh_test = train_test_split(X_val1, labels_val1, test_size=0.5, random_state=42)

In [ ]:
# Normalize data relative to the Higgs train data
x_scaler = StandardScaler()
X_hh_train = x_scaler.fit_transform(X_hh_train)
X_hh_val = x_scaler.transform(X_hh_val)
X_hh_test = x_scaler.transform(X_hh_test)
X_nohiggs = x_scaler.transform(X_nohiggs)

In [ ]:
# Split the ttbar data into train and test sets .8, .1, .1
X_tt_train, X_val1, labels_tt_train, labels_val1 = train_test_split(X_nohiggs, labels_tt, test_size=0.2, random_state=42)
X_tt_val, X_tt_test, labels_tt_val, labels_tt_test = train_test_split(X_val1, labels_val1, test_size=0.5, random_state=42)

In [ ]:
print(np.shape(X_hh_train), np.shape(X_tt_train))

In [ ]:
# Reshape the data to have shape [batch_size,particles,features]
X_hh_train = X_hh_train.reshape(X_hh_train.shape[0],6,4)
X_hh_val = X_hh_val.reshape(X_hh_val.shape[0],6,4)
X_hh_test = X_hh_test.reshape(X_hh_test.shape[0],6,4)
X_tt_train = X_tt_train.reshape(X_tt_train.shape[0],6,4)
X_tt_val = X_tt_val.reshape(X_tt_val.shape[0],6,4)
X_tt_test = X_tt_test.reshape(X_tt_test.shape[0],6,4)

# Compute some quantities to be used for calculation
phi_limit = np.max((abs(np.max(X_hh_train[:,0,2])), abs(np.min(X_hh_train[:,0,2])))) # Only need one value assuming phi distrubtions are all uniform
lower_pt_limit = [np.min(X_hh_train[:,0,0]), np.min(X_hh_train[:,1,0]), np.min(X_hh_train[:,2,0]), np.min(X_hh_train[:,3,0]), np.min(X_hh_train[:,4,0]), np.min(X_hh_train[:,5,0])]

In [ ]:
# Create a mixed sample of Higgs and ttbar data
X_train = np.append(X_hh_train, X_tt_train, axis=0)
X_val = np.append(X_hh_val, X_tt_val, axis=0)
X_test = np.append(X_hh_test, X_tt_test, axis=0)
labels_train = np.append(labels_hh_train, labels_tt_train, axis=0)
labels_val = np.append(labels_hh_val, labels_tt_val, axis=0)
labels_test = np.append(labels_hh_test, labels_tt_test, axis=0)

In [ ]:
# Load a config file from a json
with open('./configs/uci_higgs_config_demo.json', 'r') as f:
    config = json.load(f)

In [ ]:
model_name = model_name = f'Model_D{config["d_model"]}_H{config["num_heads"]}_L{config["num_layers"]}_F{config["d_ff"]}_Dr{config["dropout"]}_B{config["batch_size"]}_T{config["test_batch_size"]}_RE{config["resume_epoch"]}_NE{config["num_epochs"]}_ES{config["epochs_to_saturate"]}_IM{config["init_momentum"]}_MM{config["max_momentum"]}_ILR{config["init_lr"]}_MSL{config["max_seq_len"]}_Mk{config["mask"]}_A{config["alpha"]}_B{config["beta"]}_G{config["gamma"]}_WD{config["weight_decay"]}_MLR{config["min_lr"]}_LD{config["lr_decay"]}_CIF{config["class_input_features"]}_CFD{config["class_ff_dim"]}'

In [ ]:
# Set device to use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Build the autoencoder model
tae = pmae.TransformerAutoencoder(config['d_model'], config['num_heads'], config['num_layers'], config['d_ff'], config['max_seq_len'], config['output_vars'], config['dropout'], device)
# Build classifier
classifier = classifier.BinaryClassifier(config['class_input_features'], config['class_ff_dim'])

In [ ]:
# Send the models to gpu if possible
tae.to(device)
classifier.to(device)

In [ ]:
# Assign the criterion
criterion = utils.custom_loss(phi_limit, config['alpha'], config['beta'], config['gamma'], config['delta'], config['output_vars'])
criterion_2 = nn.BCELoss()

In [ ]:
# Instantiate the dataset and DataLoader
train_data = utils.DataLabelDataset(X_train, labels_train)
train_loader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
val_data = utils.DataLabelDataset(X_val, labels_val)
val_loader = DataLoader(val_data, batch_size=config['test_batch_size'], shuffle=True)
test_data = utils.DataLabelDataset(X_test, labels_test)
test_loader = DataLoader(test_data, batch_size=config['test_batch_size'], shuffle=False)

In [ ]:
# Build the optimizers
optimizer = utils.SGDWithSaturatingMomentumAndDecay(tae.parameters(), lr=init_lr, momentum=config['min_momentum'], max_momentum=config['max_momentum'],
                                                    epochs_to_saturate=config['epochs_to_saturate'], batches_per_epoch=len(train_loader), weight_decay=0,
                                                    lr_decay=lr_decay, min_lr=min_lr, resume_epoch=resume_epoch)
optimizer_2 = utils.SGDWithSaturatingMomentumAndDecay(list(tae.parameters())+list(classifier.parameters()), lr=init_lr, momentum=config['min_momentum'],
                                                      max_momentum=config['max_momentum'],  epochs_to_saturate=config['epochs_to_saturate'],
                                                      batches_per_epoch=len(train_loader), weight_decay=config['weight_decay'], lr_decay=config['lr_decay'],
                                                      min_lr=config['min_lr'], resume_epoch=config['resume_epoch'])

In [ ]:
train_loader, val_loader, models, device, optimizer, criterion, model_type, output_vars,

# Train the model
val_loss_min = train.train(train_loader, val_loader, [tae], device, optimizer, criterion, 'autoencoder', config['output_vars'], mask=config['mask'],
                           num_epochs=config['num_epochs']-config['resume_epoch'], save_path='./saved_models/uci_higgs', model_name=model_name)

In [ ]:
val_loss_min = train.train(train_loader, val_loader, [tae], device, optimizer_2, criterion_2, 'classifier_partial', config['output_vars'], mask=config['mask'],
                           num_epochs=config['num_epochs']-config['resume_epoch'], save_path='./saved_models/uci_higgs', model_name=model_name)

In [ ]:
val_loss_min = train.train(train_loader, val_loader, [tae], device, optimizer_2, criterion_2, 'classifier_full', config['output_vars'], mask=config['mask'],
                           num_epochs=config['num_epochs']-config['resume_epoch'], save_path='./saved_models/uci_higgs', model_name=model_name)

In [ ]:
# Parse a json config interpretation from a model name string
config = parse_model_name(model_name)

In [ ]:
# Build the autoencoder model
tae = pmae.TransformerAutoencoder(config['d_model'], config['num_heads'], config['num_layers'], config['d_ff'], config['max_seq_len'], config['output_vars'], config['dropout'], device)
# Build classifier
classifier = classifier.BinaryClassifier(config['class_input_features'], config['class_ff_dim'])

In [ ]:
tae.load_state_dict(torch.load('./saved_models/uci_higgs/TAE_best_' + model_name))

In [ ]:
# Test the model and generate some plots
classifier.load_state_dict(torch.load('./saved_models/uci_higgs/Classifier_full_best_' + model_name))
test.test(test_loader, test_batch_size, X_test, labels_test, names, [tae, classifier], mask, device, x_scaler, config['output_vars'], 'full', model_name)

In [ ]:
classifier.load_state_dict(torch.load('./saved_models/uci_higgs/Classifier_partial_best_' + model_name))
test.test(test_loader, test_batch_size, X_test, labels_test, names, [tae, classifier], mask, device, x_scaler, config['output_vars'], 'partial', model_name)